# Pools

### What we need
- Pool deposits
- Pool withdrawals
- Pool fees
- Pool volume

## Setup

In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
from mezo.currency_utils import format_token_columns
from mezo.datetime_utils import format_datetimes
from mezo.currency_config import POOL_TOKEN0_MAP
from mezo.data_utils import add_rolling_values, add_pct_change_columns, add_cumulative_columns
from mezo.clients import BigQueryClient, SubgraphClient, Web3Client

load_dotenv(dotenv_path='../.env', override=True)
COINGECKO_KEY = os.getenv('COINGECKO_KEY')

In [2]:
POOLS_SUBGRAPH = "https://api.goldsky.com/api/public/project_cm6ks2x8um4aj01uj8nwg1f6r/subgraphs/musd-pools-mezo/1.0.0/gn"

SUBGRAPH_HEADERS = {
        "Content-Type": "application/json",
    }

GET_POOL_FEES = """
query poolFees($skip: Int!) {
fees(
    first: 1000
    orderBy: timestamp_ 
    orderDirection: desc
    skip: $skip
) {
    timestamp_
    sender
    amount0
    amount1
    contractId_
    transactionHash_
    block_number
    }
}
"""

## Get pool volume data from tigris-pools-mezo

In [3]:
TIGRIS_POOLS_SUBRAPH = 'https://api.goldsky.com/api/public/project_cm6ks2x8um4aj01uj8nwg1f6r/subgraphs/tigris-pools-mezo/1.0.0/gn'

GET_POOL_VOLUME = """
query getPoolVolumes($skip: Int!) {
  poolVolumes(
    interval: day
    first: 1000
    orderBy: timestamp_ 
    orderDirection: desc
    skip: $skip
  ) {
  	timestamp
    pool {
      name
      token0 {
        symbol
      }
      token1 {
        symbol
      }
    }
    totalVolume0
    totalVolume1
  }
}
"""

In [5]:
# Instantiate subgraph
pool_volumes = SubgraphClient(
    url = TIGRIS_POOLS_SUBRAPH,
    headers = SUBGRAPH_HEADERS
)

# Fetch subgraph data
pool_volume_data =  pool_volumes.fetch_subgraph_data(GET_POOL_VOLUME, 'poolVolumes')

Fetching transactions with skip=0...
Fetching transactions with skip=1000...
No more records found.


In [4]:
def flatten_json_column(df, json_col, prefix=None):
    """
    Flatten any JSON column into separate columns using pd.json_normalize
    
    Args:
        df: DataFrame with column containing JSON data
        json_col: Name of the column containing JSON data
        prefix: Optional prefix for new column names (defaults to json_col + '_')
    
    Returns:
        DataFrame with flattened JSON data as new columns
    """
    if json_col not in df.columns:
        raise ValueError(f"Column '{json_col}' not found in DataFrame")
    
    # Set default prefix
    if prefix is None:
        prefix = f"{json_col}_"
    
    # Normalize the JSON data
    json_normalized = pd.json_normalize(df[json_col])
    
    # Create the result DataFrame starting with original data
    result_df = df.copy()
    
    # Add all flattened columns with prefix
    for col in json_normalized.columns:
        # Replace dots with underscores for cleaner column names
        clean_col_name = col.replace('.', '_')
        new_col_name = f"{prefix}{clean_col_name}"
        result_df[new_col_name] = json_normalized[col]
    
    return result_df

In [193]:
# Save pool volume data df

volume_df = pd.DataFrame(pool_volume_data)
display(volume_df)

# flatten col
volume_df_flattened = flatten_json_column(volume_df, 'pool')
volume_df_flattened = volume_df_flattened.drop(columns=['pool'])

,timestamp,pool,totalVolume0,totalVolume1
0,1754179200000000,"{'name': 'Volatile AMM - BTC/MUSD', 'token0': ...",31563459685487174,3986375566783974809470
1,1754179200000000,"{'name': 'Stable AMM - mUSDC/MUSD', 'token0': ...",3592814611,605164270302300426178
2,1754179200000000,"{'name': 'Stable AMM - MUSD/mUSDT', 'token0': ...",259931542968750000000,619634512
3,1754092800000000,"{'name': 'Volatile AMM - BTC/MUSD', 'token0': ...",94932994003387835,10564776909073260352926
4,1754092800000000,"{'name': 'Stable AMM - mUSDC/MUSD', 'token0': ...",3847371150,6440288316462755635067
...,...,...,...,...
78,1751932800000000,"{'name': 'Volatile AMM - BTC/MUSD', 'token0': ...",250000000000000,0
79,1751846400000000,"{'name': 'Volatile AMM - BTC/MUSD', 'token0': ...",0,18910000000000000000
80,1751846400000000,"{'name': 'Stable AMM - mUSDC/MUSD', 'token0': ...",2010400,0
81,1750896000000000,"{'name': 'Stable AMM - MUSD/mUSDT', 'token0': ...",0,1000000


In [194]:
volume_df_flattened.head()

# upload to bigquery

,timestamp,totalVolume0,totalVolume1,pool_name,pool_token0_symbol,pool_token1_symbol
0,1754179200000000,31563459685487174,3986375566783974809470,Volatile AMM - BTC/MUSD,BTC,MUSD
1,1754179200000000,3592814611,605164270302300426178,Stable AMM - mUSDC/MUSD,mUSDC,MUSD
2,1754179200000000,259931542968750000000,619634512,Stable AMM - MUSD/mUSDT,MUSD,mUSDT
3,1754092800000000,94932994003387835,10564776909073260352926,Volatile AMM - BTC/MUSD,BTC,MUSD
4,1754092800000000,3847371150,6440288316462755635067,Stable AMM - mUSDC/MUSD,mUSDC,MUSD


In [195]:
volume_df_flattened['pool_token0_symbol'] = volume_df_flattened['pool_token0_symbol'].replace(
    {
        'mUSDC': 'USDC', 
        'mUSDT':'USDT'
    }
)

volume_df_flattened['pool_token1_symbol'] = volume_df_flattened['pool_token1_symbol'].replace(
    {
        'mUSDC': 'USDC', 
        'mUSDT':'USDT'
    }
)

In [196]:
# ADD TO CURRENCY_CONFIG

TIGRIS_MAP = {
    'Volatile AMM - BTC/MUSD': 'btc_musd_pool',
    'Stable AMM - mUSDC/MUSD': 'musdc_musd_pool',
    'Stable AMM - MUSD/mUSDT': 'musd_musdt_pool',
    'Stable AMM - mUSDC/mUSDT': 'musdc_musdt_pool'
}

In [197]:
volume_df_flattened['pool'] = volume_df_flattened['pool_name'].map(TIGRIS_MAP)

In [198]:
vol_df_test = format_datetimes(volume_df_flattened, ['timestamp'])
vol_df_test.head()

,timestamp,totalVolume0,totalVolume1,pool_name,pool_token0_symbol,pool_token1_symbol,pool
0,2025-08-03,31563459685487174,3986375566783974809470,Volatile AMM - BTC/MUSD,BTC,MUSD,btc_musd_pool
1,2025-08-03,3592814611,605164270302300426178,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool
2,2025-08-03,259931542968750000000,619634512,Stable AMM - MUSD/mUSDT,MUSD,USDT,musd_musdt_pool
3,2025-08-02,94932994003387835,10564776909073260352926,Volatile AMM - BTC/MUSD,BTC,MUSD,btc_musd_pool
4,2025-08-02,3847371150,6440288316462755635067,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool


In [175]:
btc_musd_pool = vol_df_test.loc[vol_df_test['pool_name'] == 'Volatile AMM - BTC/MUSD']
musd_musdt_pool = vol_df_test.loc[vol_df_test['pool_name'] == 'Stable AMM - MUSD/mUSDT']
musdc_musd_pool = vol_df_test.loc[vol_df_test['pool_name'] == 'Stable AMM - mUSDC/MUSD']

In [176]:
musdc_musd_pool

,timestamp,totalVolume0,totalVolume1,pool_name,pool_token0_symbol,pool_token1_symbol,pool
1,2025-08-03,3592814611,605164270302300426178,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool
4,2025-08-02,3847371150,6440288316462755635067,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool
8,2025-08-01,3163925840,4392803091680634018801,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool
11,2025-07-31,1973092009,1769849287515544048356,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool
16,2025-07-30,811659787,1931067862080962984475,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool
20,2025-07-29,3050492516,2806068688696285906521,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool
24,2025-07-28,2154976758,1582234549152650669929,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool
28,2025-07-27,1795946009,1726251904727329401439,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool
33,2025-07-26,1468608615,1266121447936609061361,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool
35,2025-07-25,2144254648,2758516894788752981244,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool


In [188]:
musdc_musd_pool['totalVolume0'] = musdc_musd_pool['totalVolume0'].astype('float')
musdc_musd_pool['totalVolume1'] = musdc_musd_pool['totalVolume1'].astype('float')

/var/folders/9w/xx81t1c10qs96ft0v4m064800000gn/T/ipykernel_77047/2472559031.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  musdc_musd_pool['totalVolume0'] = musdc_musd_pool['totalVolume0'].astype('float')
/var/folders/9w/xx81t1c10qs96ft0v4m064800000gn/T/ipykernel_77047/2472559031.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  musdc_musd_pool['totalVolume1'] = musdc_musd_pool['totalVolume1'].astype('float')


In [189]:
musdc_musd_pool['totalVolume1'] = musdc_musd_pool['totalVolume1']/1e18

/var/folders/9w/xx81t1c10qs96ft0v4m064800000gn/T/ipykernel_77047/3222483767.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  musdc_musd_pool['totalVolume1'] = musdc_musd_pool['totalVolume1']/1e18


In [190]:
musdc_musd_pool['totalVolume0'] = musdc_musd_pool['totalVolume0']/1e6

/var/folders/9w/xx81t1c10qs96ft0v4m064800000gn/T/ipykernel_77047/692796272.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  musdc_musd_pool['totalVolume0'] = musdc_musd_pool['totalVolume0']/1e6


In [192]:
musdc_musd_pool['totalVolume0'].sum()

np.float64(81640.680299)

In [199]:
vol_df_test_2 = format_token_columns(vol_df_test, ['totalVolume0'], 'pool_token0_symbol')
vol_df_test_2.head()

,timestamp,totalVolume0,totalVolume1,pool_name,pool_token0_symbol,pool_token1_symbol,pool
0,2025-08-03,0.031563,3986375566783974809470,Volatile AMM - BTC/MUSD,BTC,MUSD,btc_musd_pool
1,2025-08-03,3592.814611,605164270302300426178,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool
2,2025-08-03,259.931543,619634512,Stable AMM - MUSD/mUSDT,MUSD,USDT,musd_musdt_pool
3,2025-08-02,0.094933,10564776909073260352926,Volatile AMM - BTC/MUSD,BTC,MUSD,btc_musd_pool
4,2025-08-02,3847.371150,6440288316462755635067,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool


In [200]:
vol_df_test_3 = format_token_columns(vol_df_test_2, ['totalVolume1'], 'pool_token1_symbol')

In [5]:
from mezo.currency_utils import get_token_prices
from mezo.currency_config import TOKENS_ID_MAP

def add_usd_conversions(df, token_column, tokens_id_map, amount_columns=None):
    """
    Add USD price conversions to any token data
    
    Args:
        df: DataFrame containing token data
        token_column: Name of column containing token identifiers
        tokens_id_map: Dictionary mapping tokens to CoinGecko IDs
        amount_columns: List of amount columns to convert, or None for auto-detection
    
    Returns:
        DataFrame with USD conversion columns added
    """
    if token_column not in df.columns:
        raise ValueError(f"Column '{token_column}' not found in DataFrame")
    
    # Get token prices
    prices = get_token_prices()
    if prices is None or prices.empty:
        raise ValueError("No token prices received from API")
    
    token_usd_prices = prices.T.reset_index()
    df_result = df.copy()
    df_result['index'] = df_result[token_column].map(tokens_id_map)
    
    df_with_usd = pd.merge(df_result, token_usd_prices, how='left', on='index')
    
    # Set MUSD price to 1.0 (1:1 with USD)
    df_with_usd.loc[df_with_usd[token_column] == 'MUSD', 'usd'] = 1.0
    
    # Auto-detect amount columns if not provided
    if amount_columns is None:
        amount_columns = [col for col in df.columns if 'amount' in col.lower() and col != 'amount_usd']
    
    # Add USD conversion for each amount column
    for col in amount_columns:
        if col in df_with_usd.columns:
            usd_col_name = f"{col}_usd" if not col.endswith('_usd') else col
            df_with_usd[usd_col_name] = df_with_usd[col] * df_with_usd['usd']
    
    return df_with_usd

In [ ]:
vol_df_test_4 = add_usd_conversions(vol_df_test_3, token_column='pool_token0_symbol', tokens_id_map=TOKENS_ID_MAP, amount_columns=['totalVolume0'])
vol_df_test_4.head()

,timestamp,totalVolume0,totalVolume1,pool_name,pool_token0_symbol,pool_token1_symbol,pool,index,usd,totalVolume0_usd
0,2025-08-03,0.031563,3986.375567,Volatile AMM - BTC/MUSD,BTC,MUSD,btc_musd_pool,bitcoin,114942.000000,3627.967183
1,2025-08-03,3592.814611,605.164270,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool,usd-coin,0.999805,3592.114012
2,2025-08-03,259.931543,619.634512,Stable AMM - MUSD/mUSDT,MUSD,USDT,musd_musdt_pool,NaN,1.000000,259.931543
3,2025-08-02,0.094933,10564.776909,Volatile AMM - BTC/MUSD,BTC,MUSD,btc_musd_pool,bitcoin,114942.000000,10911.788197
4,2025-08-02,3847.371150,6440.288316,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool,usd-coin,0.999805,3846.620913


In [203]:
vol_df_test_4['count'] = 1

In [204]:
vol_df_test_5 = vol_df_test_4[['timestamp', 'totalVolume0', 'totalVolume1', 'pool_name',
       'pool_token0_symbol', 'pool_token1_symbol', 'pool', 'totalVolume0_usd', 'count']]

In [205]:
vol_df_test_final = add_usd_conversions(vol_df_test_5, token_column='pool_token1_symbol', tokens_id_map=TOKENS_ID_MAP, amount_columns=['totalVolume1'])
vol_df_test_final.head()

,timestamp,totalVolume0,totalVolume1,pool_name,pool_token0_symbol,pool_token1_symbol,pool,totalVolume0_usd,count,index,usd,totalVolume1_usd
0,2025-08-03,0.031563,3986.375567,Volatile AMM - BTC/MUSD,BTC,MUSD,btc_musd_pool,3627.967183,1,NaN,1.000000,3986.375567
1,2025-08-03,3592.814611,605.164270,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool,3592.114012,1,NaN,1.000000,605.164270
2,2025-08-03,259.931543,619.634512,Stable AMM - MUSD/mUSDT,MUSD,USDT,musd_musdt_pool,259.931543,1,tether,0.999943,619.599193
3,2025-08-02,0.094933,10564.776909,Volatile AMM - BTC/MUSD,BTC,MUSD,btc_musd_pool,10911.788197,1,NaN,1.000000,10564.776909
4,2025-08-02,3847.371150,6440.288316,Stable AMM - mUSDC/MUSD,USDC,MUSD,musdc_musd_pool,3846.620913,1,NaN,1.000000,6440.288316


In [112]:
vol_df_test_final.columns

Index(['timestamp', 'totalVolume0', 'totalVolume1', 'pool_name',
       'pool_token0_symbol', 'pool_token1_symbol', 'pool', 'totalVolume0_usd',
       'index', 'usd', 'totalVolume1_usd'],
      dtype='object')

In [206]:
volume_final = vol_df_test_final[[
    'timestamp', 
    'totalVolume0', 'totalVolume1', 
    'pool_name', 'pool',
    'totalVolume0_usd', 'totalVolume1_usd', 'count']]

In [207]:
volume_final

,timestamp,totalVolume0,totalVolume1,pool_name,pool,totalVolume0_usd,totalVolume1_usd,count
0,2025-08-03,0.031563,3986.375567,Volatile AMM - BTC/MUSD,btc_musd_pool,3627.967183,3986.375567,1
1,2025-08-03,3592.814611,605.164270,Stable AMM - mUSDC/MUSD,musdc_musd_pool,3592.114012,605.164270,1
2,2025-08-03,259.931543,619.634512,Stable AMM - MUSD/mUSDT,musd_musdt_pool,259.931543,619.599193,1
3,2025-08-02,0.094933,10564.776909,Volatile AMM - BTC/MUSD,btc_musd_pool,10911.788197,10564.776909,1
4,2025-08-02,3847.371150,6440.288316,Stable AMM - mUSDC/MUSD,musdc_musd_pool,3846.620913,6440.288316,1
...,...,...,...,...,...,...,...,...
78,2025-07-08,0.000250,0.000000,Volatile AMM - BTC/MUSD,btc_musd_pool,28.735500,0.000000,1
79,2025-07-07,0.000000,18.910000,Volatile AMM - BTC/MUSD,btc_musd_pool,0.000000,18.910000,1
80,2025-07-07,2.010400,0.000000,Stable AMM - mUSDC/MUSD,musdc_musd_pool,2.010008,0.000000,1
81,2025-06-26,0.000000,1.000000,Stable AMM - MUSD/mUSDT,musd_musdt_pool,0.000000,0.999943,1


In [208]:
volume_by_pool = volume_final.groupby(['pool_name']).agg(
        total_volume_0 = ('totalVolume0_usd', 'sum'),
        total_volume_1 = ('totalVolume1_usd', 'sum')
    ).reset_index()

volume_by_pool

,pool_name,total_volume_0,total_volume_1
0,Stable AMM - MUSD/mUSDT,17084.837742,17795.846530
1,Stable AMM - mUSDC/MUSD,81624.760366,80727.438262
2,Stable AMM - mUSDC/mUSDT,3709.810503,3925.706027
3,Volatile AMM - BTC/MUSD,226288.313069,230842.397924


In [209]:
volume_by_pool['total_volume'] = volume_by_pool['total_volume_0'] + volume_by_pool['total_volume_1']

In [210]:
volume_by_pool

,pool_name,total_volume_0,total_volume_1,total_volume
0,Stable AMM - MUSD/mUSDT,17084.837742,17795.846530,34880.684272
1,Stable AMM - mUSDC/MUSD,81624.760366,80727.438262,162352.198629
2,Stable AMM - mUSDC/mUSDT,3709.810503,3925.706027,7635.516530
3,Volatile AMM - BTC/MUSD,226288.313069,230842.397924,457130.710993


## Get pool deposits and withdrawals

### Deposits

In [ ]:
# Fetch pool fees data
pool_fees = SubgraphClient(
    url = TIGRIS_POOLS_SUBRAPH,
    headers = SUBGRAPH_HEADERS
)

# Fetch subgraph data
pool_fees_data = pool_fees.fetch_subgraph_data(GET_TOTAL_POOL_FEES, 'feesStats_collection')

# Save pool fees data df
fees_df = pd.DataFrame(pool_fees_data)
display(fees_df)

# Flatten JSON column
fees_df_flat = flatten_json_column(fees_df, 'pool')
fees_df_flat = fees_df_flat.drop(columns=['pool'])

# Replace mUSDC and mUSDT with USDC and USDT
fees_df_flat['pool_token0_symbol'] = fees_df_flat['pool_token0_symbol'].replace({
    'mUSDC': 'USDC', 
    'mUSDT': 'USDT'
})

fees_df_flat['pool_token1_symbol'] = fees_df_flat['pool_token1_symbol'].replace({
    'mUSDC': 'USDC', 
    'mUSDT': 'USDT'
})

# Format timestamps
fees_df_formatted = format_datetimes(fees_df_flat, ['timestamp'])

# Format token columns for totalFees0 and totalFees1
fees_df_token0 = format_token_columns(fees_df_formatted, ['totalFees0'], 'pool_token0_symbol')
fees_df_token1 = format_token_columns(fees_df_token0, ['totalFees1'], 'pool_token1_symbol')

# Add USD conversions for totalFees0
fees_df_usd0 = add_usd_conversions(fees_df_token1, 'pool_token0_symbol', TOKENS_ID_MAP, ['totalFees0'])

# Add USD conversions for totalFees1 
fees_df_usd1 = add_usd_conversions(fees_df_usd0, 'pool_token1_symbol', TOKENS_ID_MAP, ['totalFees1'])

# Add count column
fees_df_usd1['count'] = 1

print("Processed fees data:")
display(fees_df_usd1.head())
print(f"\nTotal fee records: {len(fees_df_usd1)}")
print(f"Date range: {fees_df_usd1['timestamp'].min()} to {fees_df_usd1['timestamp'].max()}")

fees_df_final = fees_df_usd1

In [211]:
GET_DEPOSITS = """
    query getMints ($skip: Int!) {
    mints(
        first: 1000
        orderBy: timestamp_
        orderDirection: desc
        skip: $skip
    ) {
        timestamp_
        sender
        amount0
        amount1
        contractId_
        transactionHash_
    }
}
"""

In [212]:
# Instantiate subgraph
pool_deposits = SubgraphClient(
    url = POOLS_SUBGRAPH,
    headers = SUBGRAPH_HEADERS
)

# Fetch subgraph data
pool_deposit_data =  pool_deposits.fetch_subgraph_data(GET_DEPOSITS, 'mints')

Fetching transactions with skip=0...
Fetching transactions with skip=1000...
Fetching transactions with skip=2000...
Fetching transactions with skip=3000...
No more records found.


In [213]:
raw = pd.DataFrame(pool_deposit_data)
display(raw)

ddf = raw.copy()

,timestamp_,sender,amount0,amount1,contractId_,transactionHash_
0,1754354909,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,25769550353413,2999999999999929693,0x52e604c44417233b6ccedddc0d640a405caacefb,0xa6837b30f9444073a3826480eb9b5527c31edb2fcad0...
1,1754354642,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,3258845,2549999481900057640,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0x485c00dc328c6ad8ab976d7b6943692a6931852469ab...
2,1754348089,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,22451471253143,2599999999999920568,0x52e604c44417233b6ccedddc0d640a405caacefb,0xdb667711458c929e56ba371d5a3fc6f5738d4970a625...
3,1754346631,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,21725609696983,2511999999999998026,0x52e604c44417233b6ccedddc0d640a405caacefb,0xbed6299d4fb38690f54ae91b994a291041f359ac63eb...
4,1754346628,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,21889935566506,2530999999999956539,0x52e604c44417233b6ccedddc0d640a405caacefb,0x3eed239b86895931bdcb6f5cc776386dbada1ae296aa...
...,...,...,...,...,...,...
2486,1752047979,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,318999799072077553,35000000000000000000000,0x52e604c44417233b6ccedddc0d640a405caacefb,0xb6303d1f35ed5eb300ac2aa47111decc5e73b8581dcc...
2487,1748423531,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,50000000000000000000,50000000,0x10906a9e9215939561597b4c8e4b98f93c02031a,0xafcb120a6505c4b13593e98da5ac88b7480ac901aeeb...
2488,1748421138,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,921149594700000,100000000000000000000,0x52e604c44417233b6ccedddc0d640a405caacefb,0xa2e26a6832aaa7b0dcd39dd876c545b1bd0fd5f061f0...
2489,1748421035,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,100000000,100000000000000000000,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0x1b5b9b5e00dd53745e4de56f9d9dd701bd0bdb504fdd...


In [214]:
from mezo.currency_config import POOL_TOKEN_PAIRS, TOKENS_ID_MAP, POOLS_MAP
from mezo.currency_utils import format_pool_token_columns, add_pool_usd_conversions

ddf['pool'] = ddf['contractId_'].map(POOLS_MAP)
ddf = format_datetimes(ddf, ['timestamp_'])
ddf = format_pool_token_columns(ddf, 'contractId_', POOL_TOKEN_PAIRS)
ddf = add_pool_usd_conversions(ddf, 'contractId_', POOL_TOKEN_PAIRS, TOKENS_ID_MAP)

ddf

,timestamp_,sender,amount0,amount1,contractId_,transactionHash_,pool,token0,token1,token0_index,token1_index,token0_usd_rate,token1_usd_rate,amount0_usd,amount1_usd
0,2025-08-05,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0.000026,3.000000,0x52e604c44417233b6ccedddc0d640a405caacefb,0xa6837b30f9444073a3826480eb9b5527c31edb2fcad0...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,114964.000000,1.00000,2.962571,3.000000
1,2025-08-05,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,3.258845,2.549999,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0x485c00dc328c6ad8ab976d7b6943692a6931852469ab...,musdc_musd_pool,USDC,MUSD,usd-coin,NaN,0.999816,1.00000,3.258245,2.549999
2,2025-08-04,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0.000022,2.600000,0x52e604c44417233b6ccedddc0d640a405caacefb,0xdb667711458c929e56ba371d5a3fc6f5738d4970a625...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,114964.000000,1.00000,2.581111,2.600000
3,2025-08-04,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0.000022,2.512000,0x52e604c44417233b6ccedddc0d640a405caacefb,0xbed6299d4fb38690f54ae91b994a291041f359ac63eb...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,114964.000000,1.00000,2.497663,2.512000
4,2025-08-04,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0.000022,2.531000,0x52e604c44417233b6ccedddc0d640a405caacefb,0x3eed239b86895931bdcb6f5cc776386dbada1ae296aa...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,114964.000000,1.00000,2.516555,2.531000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2486,2025-07-09,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0.319000,35000.000000,0x52e604c44417233b6ccedddc0d640a405caacefb,0xb6303d1f35ed5eb300ac2aa47111decc5e73b8581dcc...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,114964.000000,1.00000,36673.492901,35000.000000
2487,2025-05-28,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,50.000000,50.000000,0x10906a9e9215939561597b4c8e4b98f93c02031a,0xafcb120a6505c4b13593e98da5ac88b7480ac901aeeb...,musd_musdt_pool,MUSD,USDT,NaN,tether,1.000000,0.99994,50.000000,49.997000
2488,2025-05-28,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0.000921,100.000000,0x52e604c44417233b6ccedddc0d640a405caacefb,0xa2e26a6832aaa7b0dcd39dd876c545b1bd0fd5f061f0...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,114964.000000,1.00000,105.899042,100.000000
2489,2025-05-28,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,100.000000,100.000000,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0x1b5b9b5e00dd53745e4de56f9d9dd701bd0bdb504fdd...,musdc_musd_pool,USDC,MUSD,usd-coin,NaN,0.999816,1.00000,99.981600,100.000000


In [216]:
ddf.loc[ddf['pool'] == 'musdc_musd_pool']

,timestamp_,sender,amount0,amount1,contractId_,transactionHash_,pool,token0,token1,token0_index,token1_index,token0_usd_rate,token1_usd_rate,amount0_usd,amount1_usd
1,2025-08-05,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,3.258845,2.549999,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0x485c00dc328c6ad8ab976d7b6943692a6931852469ab...,musdc_musd_pool,USDC,MUSD,usd-coin,NaN,0.999816,1.0,3.258245,2.549999
13,2025-08-04,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,3.252732,2.550000,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0x6098e1be038268250d71f682f9d9ef441c7a3b3f374a...,musdc_musd_pool,USDC,MUSD,usd-coin,NaN,0.999816,1.0,3.252133,2.550000
16,2025-08-04,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,10.595452,8.306880,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0xf6f83fcdecb22a829253ddc6f156bb4f736249f5e682...,musdc_musd_pool,USDC,MUSD,usd-coin,NaN,0.999816,1.0,10.593502,8.306880
18,2025-08-04,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,10.635673,8.338413,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0x8c3f9a73bed4d472155233143a7976f2d947185ae3b2...,musdc_musd_pool,USDC,MUSD,usd-coin,NaN,0.999816,1.0,10.633716,8.338413
19,2025-08-04,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,10.655697,8.354112,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0x2127bddfbdc93280a102a7cfdc1021732c676a298a4a...,musdc_musd_pool,USDC,MUSD,usd-coin,NaN,0.999816,1.0,10.653736,8.354112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2474,2025-07-30,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,1.000000,0.566953,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0xde2040898735fc2dad1d8ecdaefc8d50908945ba90ea...,musdc_musd_pool,USDC,MUSD,usd-coin,NaN,0.999816,1.0,0.999816,0.566953
2475,2025-07-30,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,10.000000,5.669533,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0x570ef3d70e45c586bf7b47d38905f1c1990d6106dd4c...,musdc_musd_pool,USDC,MUSD,usd-coin,NaN,0.999816,1.0,9.998160,5.669533
2482,2025-07-09,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,2500.000000,2499.975405,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0x24eb4b16b3b822087a3773e52b9875eef5a2e8c4d4b8...,musdc_musd_pool,USDC,MUSD,usd-coin,NaN,0.999816,1.0,2499.540000,2499.975405
2485,2025-07-09,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,5000.000000,4999.950809,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0xa7f442a998ca81e9b0479292a7931ee691101f79307b...,musdc_musd_pool,USDC,MUSD,usd-coin,NaN,0.999816,1.0,4999.080000,4999.950809


### Withdrawals

In [217]:
GET_WITHDRAWALS = """
query getWithdrawals($skip: Int!) {
  burns(
    first: 1000
    orderBy: timestamp_
    orderDirection: desc
    skip: $skip
) {
    timestamp_
    sender
    to
    amount0
    amount1
    contractId_
    transactionHash_
  }
}
"""

In [218]:
# Instantiate subgraph
pool_withdrawals = SubgraphClient(
    url = POOLS_SUBGRAPH,
    headers = SUBGRAPH_HEADERS
)

# Fetch subgraph data
pool_wit_data =  pool_withdrawals.fetch_subgraph_data(GET_WITHDRAWALS, 'burns')

Fetching transactions with skip=0...
Fetching transactions with skip=1000...
No more records found.


In [219]:
# Process withdrawals data in the same way as deposits
withdrawals_df = pd.DataFrame(pool_wit_data)
wdf = withdrawals_df.copy()

# Apply same processing steps as deposits
wdf['pool'] = wdf['contractId_'].map(POOLS_MAP)
wdf = format_datetimes(wdf, ['timestamp_'])
wdf = format_pool_token_columns(wdf, 'contractId_', POOL_TOKEN_PAIRS)
wdf = add_pool_usd_conversions(wdf, 'contractId_', POOL_TOKEN_PAIRS, TOKENS_ID_MAP)

print("Processed withdrawals data:")
display(wdf.head())
print(f"\nTotal withdrawals: {len(wdf)}")
print(f"Date range: {wdf['timestamp_'].min()} to {wdf['timestamp_'].max()}")

wdf

Processed withdrawals data:


,timestamp_,sender,to,amount0,amount1,contractId_,transactionHash_,pool,token0,token1,token0_index,token1_index,token0_usd_rate,token1_usd_rate,amount0_usd,amount1_usd
0,2025-08-04,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0xc85032a2e77be215a095fc8b77c0e3e16bb69bf9,5.293681,4.149987,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0xc7ef0168ab554154494a2655a4712a4f6d640664ab4f...,musdc_musd_pool,USDC,MUSD,usd-coin,NaN,0.999805,1.0,5.292649,4.149987
1,2025-08-04,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x6e80975847b9cd7d02bcf5e97e45b595b74c7999,3.010085,2.359561,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0xbf0b87272c9a092e770fa6ab08e601b03b4d1ba8f86b...,musdc_musd_pool,USDC,MUSD,usd-coin,NaN,0.999805,1.0,3.009498,2.359561
2,2025-08-04,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0xdb0e9234dec3709adb6d5c120ca1e4645873b4fb,0.000023,2.593956,0x52e604c44417233b6ccedddc0d640a405caacefb,0xc8f8f91ed0d7e695066e9252797d8469fbb2cbdc2ca7...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,114941.000000,1.0,2.601046,2.593956
3,2025-08-04,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0xe6431e2fdfe433aa785cf22ab9e1273023532d82,0.000018,2.027655,0x52e604c44417233b6ccedddc0d640a405caacefb,0x455bca65d0ac26d214fe1719f9dc1047bc51245ffb0d...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,114941.000000,1.0,2.031026,2.027655
4,2025-08-04,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x9c0f9e14b7bedae32a87be4d537c627e93ad5033,3.029276,2.371932,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0x4702f690edad4993dc5e9176d57c22f3875117e04c57...,musdc_musd_pool,USDC,MUSD,usd-coin,NaN,0.999805,1.0,3.028685,2.371932



Total withdrawals: 109
Date range: 2025-07-30 to 2025-08-04


,timestamp_,sender,to,amount0,amount1,contractId_,transactionHash_,pool,token0,token1,token0_index,token1_index,token0_usd_rate,token1_usd_rate,amount0_usd,amount1_usd
0,2025-08-04,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0xc85032a2e77be215a095fc8b77c0e3e16bb69bf9,5.293681e+00,4.149987,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0xc7ef0168ab554154494a2655a4712a4f6d640664ab4f...,musdc_musd_pool,USDC,MUSD,usd-coin,NaN,0.999805,1.000000,5.292649,4.149987
1,2025-08-04,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x6e80975847b9cd7d02bcf5e97e45b595b74c7999,3.010085e+00,2.359561,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0xbf0b87272c9a092e770fa6ab08e601b03b4d1ba8f86b...,musdc_musd_pool,USDC,MUSD,usd-coin,NaN,0.999805,1.000000,3.009498,2.359561
2,2025-08-04,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0xdb0e9234dec3709adb6d5c120ca1e4645873b4fb,2.262940e-05,2.593956,0x52e604c44417233b6ccedddc0d640a405caacefb,0xc8f8f91ed0d7e695066e9252797d8469fbb2cbdc2ca7...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,114941.000000,1.000000,2.601046,2.593956
3,2025-08-04,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0xe6431e2fdfe433aa785cf22ab9e1273023532d82,1.767016e-05,2.027655,0x52e604c44417233b6ccedddc0d640a405caacefb,0x455bca65d0ac26d214fe1719f9dc1047bc51245ffb0d...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,114941.000000,1.000000,2.031026,2.027655
4,2025-08-04,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x9c0f9e14b7bedae32a87be4d537c627e93ad5033,3.029276e+00,2.371932,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0x4702f690edad4993dc5e9176d57c22f3875117e04c57...,musdc_musd_pool,USDC,MUSD,usd-coin,NaN,0.999805,1.000000,3.028685,2.371932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,2025-07-31,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0xaeeaafb53d45cae85eef439856908dcefabc37b7,3.933180e+01,29.846236,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0xbe092f30dbb73aabf837eec62e2fc4dd7fc38670e256...,musdc_musd_pool,USDC,MUSD,usd-coin,NaN,0.999805,1.000000,39.324130,29.846236
105,2025-07-31,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x2ae80350c53bf62ac2670a6ac719cb5fa161ed64,1.090000e-04,0.000084,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0x5852a6412a1d6b3163230d5ca3d251e6b06a0d7165ab...,musdc_musd_pool,USDC,MUSD,usd-coin,NaN,0.999805,1.000000,0.000109,0.000084
106,2025-07-31,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x2ae80350c53bf62ac2670a6ac719cb5fa161ed64,5.249315e-07,0.062949,0x52e604c44417233b6ccedddc0d640a405caacefb,0x9021f36bf326a1a2be534b25e0077ee2c9fd382bebbf...,btc_musd_pool,BTC,MUSD,bitcoin,NaN,114941.000000,1.000000,0.060336,0.062949
107,2025-07-31,0x16a76d3cd3c1e3ce843c6680d6b37e9116b5c706,0x58c6a45acfcc1fd0e5a103cab2cae00b0b188ec5,2.502770e-01,0.191496,0xed812aec0fecc8fd882ac3eccc43f3aa80a6c356,0x13b577e7471b9cb5cb058540dc2f7c0f909654c11fae...,musdc_musd_pool,USDC,MUSD,usd-coin,NaN,0.999805,1.000000,0.250228,0.191496


In [223]:
wdf.loc[wdf['pool'] == 'musdc_musd_pool'].count() + 322

timestamp_          358
sender              358
to                  358
amount0             358
amount1             358
contractId_         358
transactionHash_    358
pool                358
token0              358
token1              358
token0_index        358
token1_index        322
token0_usd_rate     358
token1_usd_rate     358
amount0_usd         358
amount1_usd         358
dtype: int64

## Get pool fees

In [9]:
GET_TOTAL_POOL_FEES = """
query getFeeStats($skip: Int!) {
  feesStats_collection(
    interval: day
    first: 1000
    orderBy: timestamp
    orderDirection: desc
    skip: $skip
  ) {
    timestamp
    pool {
      name
      token0 {
        symbol
      }
      token1 {
        symbol
      }
    }
    totalFees0
    totalFees1
    id
  }
}
"""

In [10]:
# Instantiate subgraph
fees_raw = SubgraphClient(
    url = TIGRIS_POOLS_SUBRAPH,
    headers = SUBGRAPH_HEADERS
)

# Fetch subgraph data
fees_data =  fees_raw.fetch_subgraph_data(GET_TOTAL_POOL_FEES, 'feesStats_collection')

Fetching transactions with skip=0...
Fetching transactions with skip=1000...
No more records found.


In [11]:
# Save pool volume data df

fees_df = pd.DataFrame(fees_data)

# flatten col
fees_df_flat = flatten_json_column(fees_df, 'pool')
fees_df_flat = fees_df_flat.drop(columns=['pool'])

fees_df_flat.head(25)

,timestamp,totalFees0,totalFees1,id,pool_name,pool_token0_symbol,pool_token1_symbol
0,1754524800000000,417989680302547,51281557220878943244,9770560972128256,Volatile AMM - BTC/MUSD,BTC,MUSD
1,1754524800000000,503358,618330144503562745,9768155790442496,Stable AMM - mUSDC/MUSD,mUSDC,MUSD
2,1754524800000000,315404342638869525,300598,9765763493658624,Stable AMM - MUSD/mUSDT,MUSD,mUSDT
3,1754524800000000,83433,99614,9746844162719746,Stable AMM - mUSDC/mUSDT,mUSDC,mUSDT
4,1754438400000000,1095669161914945,145384564067780324094,9666506799448064,Volatile AMM - BTC/MUSD,BTC,MUSD
5,1754438400000000,39914656900000000,75704,9658711433805824,Stable AMM - MUSD/mUSDT,MUSD,mUSDT
6,1754438400000000,1958231,664423889568959672,9658196037730304,Stable AMM - mUSDC/MUSD,mUSDC,MUSD
7,1754438400000000,28410,34541,9630914405466114,Stable AMM - mUSDC/mUSDT,mUSDC,mUSDT
8,1754352000000000,598976312343821,69122841668251136570,9563006677549056,Volatile AMM - BTC/MUSD,BTC,MUSD
9,1754352000000000,1279660,298495717229805033,9562224993501184,Stable AMM - mUSDC/MUSD,mUSDC,MUSD


In [13]:
# Replace mUSDC and mUSDT with USDC and USDT
fees_df_flat['pool_token0_symbol'] = fees_df_flat['pool_token0_symbol'].replace({
    'mUSDC': 'USDC', 
    'mUSDT': 'USDT'
})

fees_df_flat['pool_token1_symbol'] = fees_df_flat['pool_token1_symbol'].replace({
    'mUSDC': 'USDC', 
    'mUSDT': 'USDT'
})

In [14]:
# Format timestamps
fees_df_flat = format_datetimes(fees_df_flat, ['timestamp'])

fees_df_flat

,timestamp,totalFees0,totalFees1,pool_name,pool_token0_symbol,pool_token1_symbol
0,2025-08-04,346886670914921,40728239247057826148,Volatile AMM - BTC/MUSD,BTC,MUSD
1,2025-08-04,548231,554307188052903998,Stable AMM - mUSDC/MUSD,USDC,MUSD
2,2025-08-04,200298097656250000,213262,Stable AMM - MUSD/mUSDT,MUSD,USDT
3,2025-08-04,0,27133,Stable AMM - mUSDC/mUSDT,USDC,USDT
4,2025-08-03,94690379056454,11959126700351924426,Volatile AMM - BTC/MUSD,BTC,MUSD
...,...,...,...,...,...,...
82,2025-07-08,750000000000,0,Volatile AMM - BTC/MUSD,BTC,MUSD
83,2025-07-07,0,56730000000000000,Volatile AMM - BTC/MUSD,BTC,MUSD
84,2025-07-07,402,0,Stable AMM - mUSDC/MUSD,USDC,MUSD
85,2025-06-26,0,100,Stable AMM - MUSD/mUSDT,MUSD,USDT


In [15]:
# Format token columns for totalFees0 and totalFees1
fees_df_flat = format_token_columns(fees_df_flat, ['totalFees0'], 'pool_token0_symbol')
fees_df_flat = format_token_columns(fees_df_flat, ['totalFees1'], 'pool_token1_symbol')

fees_df_flat

,timestamp,totalFees0,totalFees1,pool_name,pool_token0_symbol,pool_token1_symbol
0,2025-08-04,3.468867e-04,40.728239,Volatile AMM - BTC/MUSD,BTC,MUSD
1,2025-08-04,5.482310e-01,0.554307,Stable AMM - mUSDC/MUSD,USDC,MUSD
2,2025-08-04,2.002981e-01,0.213262,Stable AMM - MUSD/mUSDT,MUSD,USDT
3,2025-08-04,0.000000e+00,0.027133,Stable AMM - mUSDC/mUSDT,USDC,USDT
4,2025-08-03,9.469038e-05,11.959127,Volatile AMM - BTC/MUSD,BTC,MUSD
...,...,...,...,...,...,...
82,2025-07-08,7.500000e-07,0.000000,Volatile AMM - BTC/MUSD,BTC,MUSD
83,2025-07-07,0.000000e+00,0.056730,Volatile AMM - BTC/MUSD,BTC,MUSD
84,2025-07-07,4.020000e-04,0.000000,Stable AMM - mUSDC/MUSD,USDC,MUSD
85,2025-06-26,0.000000e+00,0.000100,Stable AMM - MUSD/mUSDT,MUSD,USDT


In [18]:
# Add USD conversions for totalFees0
fees_df_flat = add_usd_conversions(fees_df_flat, 'pool_token0_symbol', TOKENS_ID_MAP, ['totalFees0'])

# Add USD conversions for totalFees1 
fees_df_flat = add_usd_conversions(fees_df_flat, 'pool_token1_symbol', TOKENS_ID_MAP, ['totalFees1'])

# Add count column
fees_df_flat['count'] = 1

print("Processed fees data:")
display(fees_df_flat.head())
print(f"\nTotal fee records: {len(fees_df_flat)}")
print(f"Date range: {fees_df_flat['timestamp'].min()} to {fees_df_flat['timestamp'].max()}")

fees_df_final = fees_df_flat

Processed fees data:


,timestamp,totalFees0,totalFees1,pool_name,pool_token0_symbol,pool_token1_symbol,index,usd_x,totalFees0_usd,usd_y,usd,totalFees1_usd,count
0,2025-08-04,0.000347,40.728239,Volatile AMM - BTC/MUSD,BTC,MUSD,NaN,113140.000000,39.246758,NaN,1.0,40.728239,1
1,2025-08-04,0.548231,0.554307,Stable AMM - mUSDC/MUSD,USDC,MUSD,NaN,0.999742,0.548090,NaN,1.0,0.554307,1
2,2025-08-04,0.200298,0.213262,Stable AMM - MUSD/mUSDT,MUSD,USDT,tether,1.000000,0.200298,0.999764,NaN,NaN,1
3,2025-08-04,0.000000,0.027133,Stable AMM - mUSDC/mUSDT,USDC,USDT,tether,0.999742,0.000000,0.999764,NaN,NaN,1
4,2025-08-03,0.000095,11.959127,Volatile AMM - BTC/MUSD,BTC,MUSD,NaN,113140.000000,10.713269,NaN,1.0,11.959127,1



Total fee records: 87
Date range: 2025-05-28 to 2025-08-04


In [20]:
fees_df_final['totalFees1_usd'].sum()

np.float64(749.9526278609071)

In [ ]:
fees_df_final['pool'] = fees_df_final['pool_name'].map(TIGRIS_MAP)